In [1]:
!pip install pmdarima

You should consider upgrading via the '/home/ayman/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pmdarima import auto_arima
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import statsmodels.tsa.api as ts
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Bidirectional, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [3]:
tf.__version__

'2.6.0'

# Data Preparation

In [4]:
data = pd.read_csv("data/wind_temp_data.csv", parse_dates=['utc_timestamp'])
data["utc_timestamp"] = pd.to_datetime(data["utc_timestamp"])

we resample our dataset to be daily instead of hourly because otherwise the dataset is too noisy

In [5]:
data = data.resample('24H', on = 'utc_timestamp').agg({'wind_generation_actual': np.sum, 'wind_capacity': np.mean, 'temperature': np.mean})
data["utc_timestamp"] = data.index

In [6]:
data["temperature"] = data["temperature"].round(3)

In [7]:
data

In [8]:
data.info()

we have no missing data

In [9]:
data.isnull().sum()*100 / data.shape[0]

In [10]:
data.info()

In [11]:
fig, ax = plt.subplots(figsize=(8, 5))
one_year = data[(data["utc_timestamp"] > '2018-12-30 22:00:00+00:00') & (data["utc_timestamp"] <= '2019-12-30 22:00:00+00:00')]
ax.plot(one_year.index, one_year["wind_generation_actual"])
ax.grid()
ax.set_ylabel('wind generation')
ax.set_xlabel('date')
fig.tight_layout();

In [12]:
fig, ax = plt.subplots(figsize=(8, 5))
one_year = data[(data["utc_timestamp"] > '2018-12-30 22:00:00+00:00') & (data["utc_timestamp"] <= '2019-12-30 22:00:00+00:00')]
ax.plot(one_year.index, one_year["temperature"])
ax.grid()
ax.set_xlabel('date')
ax.set_ylabel('temperature variation')
fig.tight_layout();

In [13]:
fig, ax = plt.subplots(1,2,figsize=(15,8))
acf_plot = plot_acf(data["wind_generation_actual"], lags=30, ax = ax[0])
pacf_plot = plot_pacf(data["wind_generation_actual"], lags=50, ax = ax[1])
plt.show()

these plots can help us determine the ARIMAX(p,d,q) model parameters where p is determined by acf plot and q is determined by partial acf plot and also help in feature engineering

In [14]:
def correlation_heatmap(df):
    _ , ax = plt.subplots(figsize =(14, 12))
    colormap = sns.diverging_palette(220, 10, as_cmap = True)
    
    _ = sns.heatmap(
        df.corr(), 
        cmap = colormap,
        square=True, 
        cbar_kws={'shrink':.9 }, 
        ax=ax,
        annot=True, 
        linewidths=0.1,vmax=1.0, linecolor='white',
        annot_kws={'fontsize':12 }
    )
    
    plt.title('Pearson Correlation of Features', y=1.05, size=15)

correlation_heatmap(data)

# Feature engineering

we create new features such as year or month and shifted versions of our original features (lag features)

In [15]:
def feature_engineering(data):
    
    data["utc_timestamp"] = pd.to_datetime(data["utc_timestamp"], format="%Y-%m-%d")
    data["year"] = data["utc_timestamp"].dt.year
    data["month"] = data["utc_timestamp"].dt.month
    
    lag_features = ["wind_generation_actual", "temperature", "wind_capacity"]
    window1 = 2
    window2 = 3
    window3 = 7

    data_rolled_2d = data[lag_features].rolling(window=window1, min_periods=0)
    data_rolled_3d = data[lag_features].rolling(window=window2, min_periods=0)
    data_rolled_7d = data[lag_features].rolling(window=window3, min_periods=0)

    data_mean_2d = data_rolled_2d.mean().shift(1)
    data_mean_3d = data_rolled_3d.mean().shift(1)
    data_mean_7d = data_rolled_7d.mean().shift(1)
    
    data_std_2d = data_rolled_2d.std().shift(1)
    data_std_3d = data_rolled_3d.std().shift(1)
    data_std_7d = data_rolled_7d.std().shift(1)

    for feature in lag_features:
        data[f"{feature}_mean_lag{window1}"] = data_mean_2d[feature]
        data[f"{feature}_mean_lag{window2}"] = data_mean_3d[feature]
        data[f"{feature}_mean_lag{window3}"] = data_mean_7d[feature]

        data[f"{feature}_std_lag{window1}"] = data_std_2d[feature]
        data[f"{feature}_std_lag{window2}"] = data_std_3d[feature]
        data[f"{feature}_std_lag{window3}"] = data_std_7d[feature]

    data.fillna(data.mean(), inplace=True)
    
feature_engineering(data)

In [16]:
data

In [17]:
data.isnull().sum()

In [18]:
def correlation_heatmap(df):
    _ , ax = plt.subplots(figsize =(20, 15))
    colormap = sns.diverging_palette(220, 10, as_cmap = True)
    
    _ = sns.heatmap(
        df.corr(), 
        cmap = colormap,
        square=True, 
        cbar_kws={'shrink':.9 }, 
        ax=ax,
        annot=True, 
        linewidths=0.1,vmax=1.0, linecolor='white',
        annot_kws={'fontsize':12 }
    )
    
    plt.title('Pearson Correlation of Features', y=1.05, size=15)

correlation_heatmap(data)

we split our data into train and test, train being from 2017 to 2019 and test being from 2019 onwards

In [19]:
train = data[data.year < 2019]
valid = data[data.year >= 2019]

In [20]:
train.columns

the adfuller test is used to determine if our time series is stationary or not, here we see that it's stationary

In [21]:
result = adfuller(data["wind_generation_actual"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

In [22]:
data.columns

In [23]:
features = ['temperature', 'wind_capacity', 'year', 'month', 'wind_generation_actual_mean_lag2',
            'wind_generation_actual_mean_lag3', 'wind_generation_actual_mean_lag7',
            'wind_generation_actual_std_lag2', 'wind_generation_actual_std_lag3',
            'wind_generation_actual_std_lag7', 'temperature_mean_lag2',
            'temperature_mean_lag3', 'temperature_mean_lag7',
            'temperature_std_lag2', 'temperature_std_lag3', 'temperature_std_lag7',
           'wind_capacity_mean_lag2', 'wind_capacity_mean_lag3',
           'wind_capacity_mean_lag7', 'wind_capacity_std_lag2',
           'wind_capacity_std_lag3', 'wind_capacity_std_lag7']

In [24]:
len(features)

# ARIMAX

In [25]:
arimaxmodel = auto_arima(train['wind_generation_actual'],
                   exogenous = train[features], trace=True, 
                   error_action="ignore", suppress_warnings=True)
arimaxmodel.fit(train['wind_generation_actual'], exogenous = train[features])
# model = arima()
arimaxforecast = arimaxmodel.predict(n_periods=len(valid), exogenous = valid[features])
valid["Forecast_ARIMAX"] = arimaxforecast

In [26]:
arimaxmodel.summary()

In [27]:
valid[["wind_generation_actual", "Forecast_ARIMAX"]].plot(figsize=(14, 7))

In [68]:
arimax_rmse = np.sqrt(mean_squared_error(valid["wind_generation_actual"], valid.Forecast_ARIMAX))
arimax_mae = mean_absolute_error(valid["wind_generation_actual"], valid.Forecast_ARIMAX)

print("RMSE of Auto ARIMAX:", arimax_rmse)
print("\nMAE of Auto ARIMAX:", arimax_mae)

In [29]:
fig, axes = plt.subplots(figsize=(15, 10))

axes.plot(train.iloc[350:,:]["wind_generation_actual"], label='Training Data')
axes.plot(valid.index, valid["wind_generation_actual"], label='Actual Values')
axes.plot(valid.index, arimaxforecast,label='Predicted Values')
axes.set_title('Wind energy forecasts')
axes.set_xlabel('Dates')
axes.set_ylabel('Energy production')
axes.legend()

# Decision Trees

In [30]:
dtregressor = DecisionTreeRegressor()
dtregressor.fit(train[features], train['wind_generation_actual'])
dtforecast = dtregressor.predict(valid[features])
valid["Forecast_DT"] = dtforecast

In [31]:
valid[["wind_generation_actual", "Forecast_DT"]].plot(figsize=(14, 7))

In [32]:
dt_rmse = np.sqrt(mean_squared_error(valid["wind_generation_actual"], valid.Forecast_DT))
dt_mae = mean_absolute_error(valid["wind_generation_actual"], valid.Forecast_DT)

print("RMSE of Decision Trees:", dt_rmse)
print("\nMAE of Decision Trees:", dt_mae)

In [33]:
fig, axes = plt.subplots(figsize=(15, 10))

axes.plot(train.iloc[350:,:]["wind_generation_actual"], label='Training Data')
axes.plot(valid.index, valid["wind_generation_actual"], label='Actual Values')
axes.plot(valid.index, dtforecast,label='Predicted Values')
axes.set_title('Wind energy forecasts')
axes.set_xlabel('Dates')
axes.set_ylabel('Energy production')
axes.legend()

# Random Forest

In [34]:
rfregressor = RandomForestRegressor()
rfregressor.fit(train[features], train['wind_generation_actual'])
rfforecast = rfregressor.predict(valid[features])
valid["Forecast_RF"] = rfforecast

In [35]:
valid[["wind_generation_actual", "Forecast_RF"]].plot(figsize=(14, 7))

In [36]:
rf_rmse = np.sqrt(mean_squared_error(valid["wind_generation_actual"], valid.Forecast_RF))
rf_mae = mean_absolute_error(valid["wind_generation_actual"], valid.Forecast_RF)

print("RMSE of Random Forests:", rf_rmse)
print("\nMAE of Random Forests:", rf_mae)

In [37]:
fig, axes = plt.subplots(figsize=(15, 10))

axes.plot(train.iloc[350:,:]["wind_generation_actual"], label='Training Data')
axes.plot(valid.index, valid["wind_generation_actual"], label='Actual Values')
axes.plot(valid.index, rfforecast,label='Predicted Values')
axes.set_title('Wind energy forecasts')
axes.set_xlabel('Dates')
axes.set_ylabel('Energy production')
axes.legend()

# Support Vector Machine

In [38]:
svmregressor = SVR(kernel = "linear")
svmregressor.fit(train[features], train['wind_generation_actual'])
svmforecast = svmregressor.predict(valid[features])
valid["Forecast_SVM"] = svmforecast

In [39]:
valid[["wind_generation_actual", "Forecast_SVM"]].plot(figsize=(14, 7))

In [40]:
svm_rmse = np.sqrt(mean_squared_error(valid["wind_generation_actual"], valid.Forecast_SVM))
svm_mae = mean_absolute_error(valid["wind_generation_actual"], valid.Forecast_SVM)

print("RMSE of SVM:", svm_rmse)
print("\nMAE of SVM:", svm_mae)

In [41]:
fig, axes = plt.subplots(figsize=(15, 10))

axes.plot(train.iloc[350:,:]["wind_generation_actual"], label='Training Data')
axes.plot(valid.index, valid["wind_generation_actual"], label='Actual Values')
axes.plot(valid.index, svmforecast,label='Predicted Values')
axes.set_title('Wind energy forecasts')
axes.set_xlabel('Dates')
axes.set_ylabel('Energy production')
axes.legend()

# Artificial Neural Network

In [42]:
X_train, X_valid, Y_train, Y_valid=train[features], valid[features], train["wind_generation_actual"], valid["wind_generation_actual"]
X_train, X_valid = np.array(X_train), np.array(X_valid)

X_train_array = np.asarray(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])))
X_valid_array = np.asarray(X_valid.reshape(X_valid.shape[0], 1, X_valid.shape[1]))
Y_train_array = np.asarray(Y_train) 
Y_valid_array = np.asarray(Y_valid)

In [43]:
X_train.shape

In [44]:
X_valid_array.shape

In [186]:
ann_model = Sequential([
    Dense(32, activation="relu", input_shape=(1, X_train_array.shape[2])),
    Dense(8, activation="relu"),
    Dense(1)
    ])

In [223]:
ann_model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(learning_rate = 0.001),
                metrics=[tf.metrics.RootMeanSquaredError()])

In [225]:
history = ann_model.fit(X_train_array, Y_train_array, 
     validation_data=(X_valid_array, Y_valid_array),
     epochs=150, verbose=1, batch_size = 128)

In [226]:
annforecast = ann_model.predict(X_valid_array)
valid["Forecast_ANN"] = annforecast.squeeze()

In [227]:
valid[["wind_generation_actual", "Forecast_ANN"]].plot(figsize=(14, 7))

In [228]:
ann_rmse = np.sqrt(mean_squared_error(valid["wind_generation_actual"], valid.Forecast_ANN))
ann_mae = mean_absolute_error(valid["wind_generation_actual"], valid.Forecast_ANN)

print("RMSE of ANN:", ann_rmse)
print("\nMAE of ANN:", ann_mae)

In [192]:
fig, axes = plt.subplots(figsize=(15, 10))

axes.plot(train.iloc[350:,:]["wind_generation_actual"], label='Training Data')
axes.plot(valid.index, valid["wind_generation_actual"], label='Actual Values')
axes.plot(valid.index, annforecast.squeeze(), label='Predicted Values')
axes.set_title('Wind energy forecasts')
axes.set_xlabel('Dates')
axes.set_ylabel('Energy production')
axes.legend()

# Recurrent Neural Network

In [217]:
rnn_model = Sequential([
    SimpleRNN(50, activation = "relu", return_sequences=False, input_shape=(1, X_train_array.shape[2])),
    Dense(8, activation="relu"),
    Dense(1)

    ])
rnn_model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(learning_rate = 0.001),
                metrics=[tf.metrics.RootMeanSquaredError()])

In [229]:
history = rnn_model.fit(X_train_array, Y_train_array, 
     validation_data=(X_valid_array, Y_valid_array),
     epochs=150, verbose=2, batch_size = 128)

In [230]:
rnnforecast = rnn_model.predict(X_valid_array)
valid["Forecast_RNN"] = rnnforecast

In [231]:
valid[["wind_generation_actual", "Forecast_RNN"]].plot(figsize=(14, 7))

In [232]:
rnn_rmse = np.sqrt(mean_squared_error(valid["wind_generation_actual"], valid.Forecast_RNN))
rnn_mae = mean_absolute_error(valid["wind_generation_actual"], valid.Forecast_RNN)

print("RMSE of RNN:", rnn_rmse)
print("\nMAE of RNN:", rnn_mae)

In [233]:
fig, axes = plt.subplots(figsize=(15, 10))

axes.plot(train.iloc[350:,:]["wind_generation_actual"], label='Training Data')
axes.plot(valid.index, valid["wind_generation_actual"], label='Actual Values')
axes.plot(valid.index, rnnforecast,label='Predicted Values')
axes.set_title('Wind energy forecasts')
axes.set_xlabel('Dates')
axes.set_ylabel('Energy production')
axes.legend()

# LSTM

In [234]:
lstm_model = Sequential([
    Bidirectional(LSTM(100, activation = "relu", return_sequences=False, input_shape=(1, X_train_array.shape[2]))),
    Dense(8, activation="relu"),
    Dense(1)

    ])
lstm_model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(learning_rate = 0.001),
                metrics=[tf.metrics.RootMeanSquaredError()])

In [235]:
history = lstm_model.fit(X_train_array, Y_train_array, 
     validation_data=(X_valid_array, Y_valid_array),
     epochs=150, verbose=2, batch_size = 128)

In [236]:
lstmforecast = lstm_model.predict(X_valid_array)
valid["Forecast_LSTM"] = lstmforecast

In [237]:
valid[["wind_generation_actual", "Forecast_LSTM"]].plot(figsize=(14, 7))

In [238]:
lstm_rmse = np.sqrt(mean_squared_error(valid["wind_generation_actual"], valid.Forecast_LSTM))
lstm_mae = mean_absolute_error(valid["wind_generation_actual"], valid.Forecast_LSTM)

print("RMSE of LSTM:", lstm_rmse)
print("\nMAE of LSTM:", lstm_mae)

In [239]:
fig, axes = plt.subplots(figsize=(15, 10))

axes.plot(train.iloc[350:,:]["wind_generation_actual"], label='Training Data')
axes.plot(valid.index, valid["wind_generation_actual"], label='Actual Values')
axes.plot(valid.index, lstmforecast,label='Predicted Values')
axes.set_title('Wind energy forecasts')
axes.set_xlabel('Dates')
axes.set_ylabel('Energy production')
axes.legend()

# Conclusion

In [240]:

metrics = {'Models' : ['ARIMAX', 'Decision Trees', 'Random Forest', 'SVM', 'ANN', 'RNN', 'LSTM'],
           'RMSE'   : [arimax_rmse, dt_rmse, rf_rmse, svm_rmse, ann_rmse, rnn_rmse, lstm_rmse],
           'MAE'    : [arimax_mae, dt_mae, rf_mae, svm_mae, ann_mae, rnn_mae, lstm_mae],
           'NRMSE'  : [arimax_rmse/valid.Forecast_ARIMAX.mean(), dt_rmse/valid.Forecast_DT.mean(), 
                       rf_rmse/valid.Forecast_RF.mean(), svm_rmse/valid.Forecast_SVM.mean(),
                       ann_rmse/valid.Forecast_ANN.mean(), rnn_rmse/valid.Forecast_RNN.mean(),
                       lstm_rmse/valid.Forecast_LSTM.mean()]}
metrics = pd.DataFrame(metrics)

In [241]:
metrics.sort_values(by = 'RMSE')

In [242]:
metrics.sort_values(by = 'MAE')